In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
# 데이터 정보
data_schema = pd.read_csv('./TC_ELCTY_ATMBL_ELCTC_RECD_20230609095252_schema.csv', encoding='euc-kr')
data_schema

,영문 컬럼명,한글 컬럼명,컬럼 타입,컬럼 길이,NULL 여부
0,ELCTC_DTM,충전_일시,VARCHAR2,19.0,True
1,CHRSTN_ID,충전소_ID,VARCHAR2,20.0,True
2,CHRGR_ID,충전기_ID,VARCHAR2,20.0,True
3,CTPR_CD,시도_코드,VARCHAR,10.0,True
4,SIGNGU_CD,시군구_코드,VARCHAR,5.0,True
5,CTPR_NM,시도_명,VARCHAR2,50.0,True
6,SIGNGU_NM,시군구_명,VARCHAR2,50.0,True
7,CHRSTN_NM,충전소_명,VARCHAR2,50.0,True
8,LA,위도,NUMBER,1310.0,False
9,LO,경도,NUMBER,1310.0,False


In [3]:
# 충전기ID, 시작일시, 종료일시, 전력사용량만 활용한다
# 충전기ID 별로 데이터를 정리한다
# 충전 속도는 일정하다고 가정한다
# 충전을 1시 30분부터 2시 30분까지 했다면 충전량을 30:30으로 나눠서 1시 데이터에 누적, 2시데이터에 누적한다
# 데이터를 기록한 총 날수로 나눠서 각 충전기ID의 시간별 충전량 평균치를 구한다
# 이 값으로 어떻게든 클러스터링 한다

In [4]:
# 데이터 목록 가져오기
path = "./TC_ELCTY_ATMBL_ELCTC_RECD_20230609095252/"
file_list = os.listdir(path)

In [5]:
# 데이터 가져오기
data = []
for file_name in file_list:
    file_path = path + file_name
    data.append(pd.read_csv(file_path)[['충전기_ID', '시작_일시', '종료_일시', '전력_사용량']])

data = pd.concat(data, ignore_index=True)

In [6]:
# 가져온 데이터 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11094788 entries, 0 to 11094787
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   충전기_ID  object
 1   시작_일시   object
 2   종료_일시   object
 3   전력_사용량  int64 
dtypes: int64(1), object(3)
memory usage: 338.6+ MB


In [7]:
# 첫 데이터로 데이터 형식 확인
data.head(1)

,충전기_ID,시작_일시,종료_일시,전력_사용량
0,KRPPKCP0002,2022-12-31 22:12:28,2023-01-01 00:00:00,140


In [8]:
# 날짜 데이터에서 시분초 데이터만 추출해서 추가
data['시작_시간'] = (data['시작_일시'].str[-8:-6] + data['시작_일시'].str[-5:-3] + data['시작_일시'].str[-2:])
data['종료_시간'] = (data['종료_일시'].str[-8:-6] + data['종료_일시'].str[-5:-3] + data['종료_일시'].str[-2:])

In [9]:
# 날짜 데이터 삭제
data = data.drop(['시작_일시', '종료_일시'], axis=1)

In [10]:
# 바꾼 데이터 확인
data.head(1)

,충전기_ID,전력_사용량,시작_시간,종료_시간
0,KRPPKCP0002,140,221228,000000


In [48]:
# 충전기 id를 사전순으로 정렬한 것
ids = sorted(data['충전기_ID'].unique())

In [55]:
# 충전량 데이터
# 가로축 충전소 ID, 세로축 시각
id_time_charge = {}
for i in ids:
    id_time_charge[i] = np.zeros(24)

In [50]:
# 문자열 시간 데이터에서 시, 분, 초를 정수형으로 반환하는 함수
def getHMS(t):
    # '030000'을 정수형으로 바꾸면 030000이 아니라 30000이 되므로 뒤에서 부터 인덱싱 한다
    tH = int(t[:-4]) # 시
    tM = int(t[-4:-2]) # 분
    tS = int(t[-2:]) # 초
    return tH, tM, tS

In [51]:
# 두개의 문자열 시간 데이터의 차이를 초 단위로 반환하는 함수
def cal_dt(s,e):
    # 시분초 구하기
    sH, sM, sS = getHMS(s)
    eH, eM, eS = getHMS(e)

    # 시작 시간이 종료 시간보다 크면 종료시간에 24를 더하기
    if sH > eH:
        eH += 24

    # 시분초 각각의 차이 계산
    dH = eH-sH
    dM = eM-sM
    dS = eS-sS

    # 시*3600 + 분*60 + 초
    return dH*3600 + dM*60 + dS

In [ ]:
# 각각의 충전기의 시간별 데이터 입력

for i in range(len(data)):
    d = data.iloc[i] # 1개의 데이터 가져오기 
    
    id = d['충전기_ID'] # ID
    s = d['시작_시간'] # start time
    e = d['종료_시간'] # end time
    c = d['전력_사용량'] # charge

    if c == 0:
        continue

    # 문자열 시간데이터 정수형으로 바꾸기
    sH, sM, sS = getHMS(s)
    eH, eM, eS = getHMS(e)

    # 시작 시간이 종료 시간보다 크면 종료시간에 24를 더하기
    if sH > eH:
        eH += 24

    # 총 충전시간 구하기
    dt = cal_dt(s,e)
    if dt <= 0:
        continue

    # 정각시간문자열
    # 정각시간을 문자열로 구하기 위해 정의
    o = '0000'


    # 각 시간대의 충전량을 구하기
    # 소수점 아래는 버린다
    
    # 시작 시간과 다음 정각과의 시간차를 구한 뒤
    # 충전 시간과의 비율을 구해 전력 사용량과 곱해
    # 시간대의 충전량을 구하기
    id_time_charge[id][int(s[:2])] += cal_dt(s, str(sH+1)+o)*c//dt

    # 시작 시간, 종료 시간이 아니라면 1시간 내내 충전하므로
    # 시간차를 구하지 않고 3600으로 계산한다
    for i in range(sH+1, eH):
        id_time_charge[id][i%24] += 3600*c//dt

    # 끝 시간과 그 전 정각과의 시간차를 구한 뒤
    # 충전 시간과의 비율을 구해 전력 사용량과 곱해
    # 시간대의 충전량을 구하기
    if eM + eS == 0:
        continue
    id_time_charge[id][int(e[:2])] += cal_dt(str(eH)+o, e)*c//dt

In [ ]:
# 2023년 1월 1일부터 3월 31일 까지 데이터 이므로
# 하루당 데이터를 얻으려면 

# 데이터 기록 후 데이터프레임으로 변환
# 충전량 데이터
# 가로축 시각, 세로축 충전소 ID
id_time_charge_df = pd.DataFrame(id_time_charge).T

In [ ]:
id_time_charge_df